In [1]:
# Import required libraries
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import os
import dash_mantine_components as dmc
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_table
%pip install dask
import dask.dataframe as dd

/Users/vishalsehgal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/hd/p1cpt0jj6z5_ngrvf3vvz4tr0000gn/T/ipykernel_3041/3180242205.py:10: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/hd/p1cpt0jj6z5_ngrvf3vvz4tr0000gn/T/ipykernel_3041/3180242205.py:11: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/hd/p1cpt0jj6z5_ngrvf3vvz4tr0000gn/T/ipykernel_3041/3180242205.py:13: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


/Users/vishalsehgal/Library/Python/3.9/lib/python/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Main Feature
1. Players performance coomparision : Compare players from different clubs when they face each other in their national teams during the World Cup.
2. Team performance comparision : Analyze team performance and how well players know each other's strengths and weaknesses.


Steps:
1. Integrate Match Data:    Use the Fifa_Worldcup_2022_Groups.csv for match data.
2. Filter Players Based on Match Selection:     Allow users to filter players by match, country, and club.
3. Visual Comparison:   Display a visual comparison of player performance.

# Get Paths and Data

In [2]:
# Initialize the Dash app
app = JupyterDash(__name__,
        external_stylesheets=[
            # include google fonts
            "https://fonts.googleapis.com/css2?family=Inter:wght@100;200;300;400;500;900&display=swap"
        ],
)

/Users/vishalsehgal/Library/Python/3.9/lib/python/site-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [3]:
server = app.server # Needed for gunicorn

In [4]:
player_data_file = 'assets/data/FIFA22_official_data.csv'
match_data_file = 'assets/data/Matchdata.csv'
group_data_file = 'assets/data/Fifa_Worldcup_2022_Groups.csv'
group_df = pd.read_csv(group_data_file)
player_df = pd.read_csv(player_data_file)
match_df = pd.read_csv(match_data_file)
match_df.columns.tolist()
group_df.columns.tolist()
#match_df.head()

['Group', 'Flag_Image', 'Country_Name', 'Country_Name_Short']

In [5]:
merged_df = match_df.merge(group_df, left_on='home_team', right_on='Country_Name', how='left')
merged_df = merged_df.merge(group_df, left_on='away_team', right_on='Country_Name', how='left', suffixes=('_home', '_away'))
merged_with_home_players_ddf = dd.from_pandas(merged_df, npartitions=4)
player_ddf = dd.from_pandas(player_df, npartitions=4)
final_merged_ddf = dd.merge(
    merged_with_home_players_ddf,
    player_ddf,
    left_on='Country_Name_away',
    right_on='Nationality',
    how='left',
    suffixes=('_home', '_away')
)
final_merged_df = final_merged_ddf.compute()

# Get the list of column names from the final merged DataFrame
columns_list = final_merged_df.columns.tolist()

# Output the column names
columns_list

['match',
 'dayofweek',
 'match_time',
 'home_team',
 'away_team',
 'home_xg',
 'away_xg',
 'score',
 'attendance',
 'venue',
 'referee',
 'home_formation',
 'away_formation',
 'home_captain',
 'away_captain',
 'home_manager',
 'away_manager',
 'home_possession',
 'away_possession',
 'home_completed_passes',
 'home_attempted_pases',
 'away_completed_passes',
 'away_attempted_pases',
 'home_sot',
 'away_sot',
 'home_total_shots',
 'away_total_shots',
 'home_saves',
 'away_saves',
 'home_fouls',
 'away_fouls',
 'home_corners',
 'away_corners',
 'home_crosses',
 'away_crosses',
 'home_touches',
 'away_touches',
 'home_tackles',
 'away_tackles',
 'home_interceptions',
 'away_interceptions',
 'home_aerials_won',
 'away_aerials_won',
 'home_clearances',
 'away_clearances',
 'home_offsides',
 'away_offsides',
 'home_gks',
 'away_gks',
 'home_throw_ins',
 'away_throw_ins',
 'home_long_balls',
 'away_long_balls',
 'Group_home',
 'Flag_Image_home',
 'Country_Name_home',
 'Country_Name_Short_home

In [6]:
player_stats = player_df[['Name', 'Nationality', 'Overall', 'Club', 
                          'Crossing', 'Finishing', 'HeadingAccuracy', 
                          'ShortPassing', 'Volleys', 'Dribbling', 
                          'Curve', 'FKAccuracy', 'LongPassing', 
                          'BallControl', 'Acceleration', 'SprintSpeed', 
                          'Agility', 'Reactions', 'Balance', 'ShotPower', 
                          'Jumping', 'Stamina', 'Strength', 'LongShots', 
                          'Aggression', 'Interceptions', 'Positioning', 
                          'Vision', 'Penalties', 'Composure']]

unique_matches = match_df.apply(lambda x: f"{x['home_team']} vs {x['away_team']}", axis=1).unique()
unique_players = player_stats['Name'].unique()


In [7]:
app.layout = html.Div([
    # Header Section
    html.Header([
        html.Img(src=app.get_asset_url('Worldcuplogo.png'), className='header-logo'),  # Logo
        html.Div([
            html.H1('FIFA SHOWDOWN', className='header-title'),
            html.H2('Comparing Player Performance Across Teams', className='header-content')
        ], className='header-content'),
        html.Img(src=app.get_asset_url('football.png'), className='header-logo-ball'),
        html.Img(src=app.get_asset_url('something.png'), className='header-logo-something')
    ], className='header'),

    # Dropdown for Match Selection
    html.Div([
        dcc.Dropdown(
            id='match-dropdown',
            options=[{'label': match, 'value': match} for match in unique_matches],
            value=None,
            placeholder="Select Match",
            className='dropdown'
        )
    ]),
    
    # Dropdowns for Player Selection
    html.Div([
        dcc.Dropdown(
            id='player1-dropdown',
            options=[],
            value=None,
            placeholder="Select Player 1",
            className='dropdown'
        ),
        dcc.Checklist(
            id='same-club-checkbox',
            options=[{'label': 'Choose Players from the Same Club', 'value': 'same_club'}],
            value=[],
            className='checkbox'
        ),
        html.Br(),
        dcc.Dropdown(
            id='player2-dropdown',
            options=[],
            value=None,
            placeholder="Select Player 2",
            className='dropdown'
        ),
    ], className='dropdown-container'),

    # Container for the card components
    html.Div(
        className="container",
        children=[
            # Left Segment for Player 1
            html.Div(
                className="segment left_segment",
                children=[
                    html.Div(
                        className='left_card_div',
                        children=[
                            html.Div(
                                className='cardcomponent',
                                children=[
                                    html.Div(id='player1-header', className='header'),  # Player 1 Header
                                    html.Div(
                                        className='content',
                                        children=[
                                            html.Div(
                                                className='picture-container',
                                                children=[
                                                    html.Img(id='player1-image', className='player-image', src='https://cdn.sofifa.net/players/158/023/22_60.png')
                                                ]
                                            ),
                                            dash_table.DataTable(
                                                id='stats-table1',
                                                columns=[
                                                    {"name": "Stat", "id": "stat"},
                                                    {"name": "Value", "id": "value"}
                                                ],
                                                data=[],
                                                style_table={'overflowX': 'auto'},
                                                style_header={'backgroundColor': '#f1f1f1'},
                                                style_cell={'textAlign': 'left', 'padding': '8px', 'border': '1px solid #ddd'},
                                                style_as_list_view=True
                                            )
                                        ]
                                    )
                                ]
                            )
                        ]
                    )
                ]
            ),
            # Middle Segment for Graph
            html.Div(
                className="segment middle_segment",
                children=[
                    dcc.Graph(id='radar-plot'),
                    dcc.Graph(id='scatter-plot'),
                    html.Div(
                        className='dropdown-row',
                        children=[
                            dcc.Dropdown(
                                id='scatter-x-axis',
                                options=[{'label': player_stats.columns[2], 'value': player_stats.columns[2]}] + [{'label': stat, 'value': stat} for stat in player_stats.columns[4:]],
                                value='Overall',
                                placeholder="Select X-axis",
                                className='dropdown'
                            ),
                            dcc.Dropdown(
                                id='scatter-y-axis',
                                options=[{'label': player_stats.columns[2], 'value': player_stats.columns[2]}] + [{'label': stat, 'value': stat} for stat in player_stats.columns[4:]],
                                value='Reactions',
                                placeholder="Select Y-axis",
                                className='dropdown'
                            ),
                        ],
                    ),

                    html.Div(
                        className='graph-row',
                        children=[
                            dcc.Graph(id='boxplot-player1', className='graph'),
                            dcc.Graph(id='boxplot-player2', className='graph')
                        ]
                    ),
                    html.Div(
                        className='dropdown-row',
                        children=[
                            dcc.Dropdown(
                                id='stat-dropdown',
                                options=[{'label': player_stats.columns[2], 'value': player_stats.columns[2]}] + [{'label': stat, 'value': stat} for stat in player_stats.columns[4:]],
                                value=player_df.columns[2],  # Default value
                                placeholder="Select Stat",
                                className='dropdown'
                            ),
                        ],
                    )
                ]
            ),
            # Right Segment for Player 2
            html.Div(
                className="segment right_segment",
                children=[
                    html.Div(
                        className='right_card_div',
                        children=[
                            html.Div(
                                className='cardcomponent',
                                children=[
                                    html.Div(id='player2-header', className='header'),  # Player 2 Header
                                    html.Div(
                                        className='content',
                                        children=[
                                            html.Div(
                                                className='picture-container',
                                                children=[
                                                    html.Img(id='player2-image', className='player-image', src='https://cdn.sofifa.net/players/020/801/22_60.png')
                                                ]
                                            ),
                                            dash_table.DataTable(
                                                id='stats-table2',
                                                columns=[
                                                    {"name": "Stat", "id": "stat"},
                                                    {"name": "Value", "id": "value"}
                                                ],
                                                data=[],
                                                style_table={'overflowX': 'auto'},
                                                style_header={'backgroundColor': '#f1f1f1'},
                                                style_cell={'textAlign': 'left', 'padding': '8px', 'border': '1px solid #ddd'},
                                                style_as_list_view=True
                                            )
                                        ]
                                    )
                                ]
                            )
                        ]
                    )
                ]
            )
        ]
    )
])


In [8]:
# Define the callback to update player dropdowns based on selected match
@app.callback(
    [Output('player1-dropdown', 'options'),
     Output('player2-dropdown', 'options')],
    [Input('match-dropdown', 'value'),
     Input('player1-dropdown', 'value'),
     Input('same-club-checkbox', 'value')]
)
def update_player_dropdowns(selected_match, player1, same_club):
    if selected_match is None:
        return [], []

    home_team, away_team = selected_match.split(' vs ')
    players_in_match = player_stats[player_stats['Nationality'].isin([home_team, away_team])]

    player1_options = [{'label': player, 'value': player} for player in players_in_match['Name'].unique()]

    if player1 is not None and 'same_club' in same_club:
        player1_club = player_stats[player_stats['Name'] == player1]['Club'].values[0]
        players_in_match = players_in_match[players_in_match['Club'] == player1_club]

    player2_options = [{'label': player, 'value': player} for player in players_in_match['Name'].unique() if player != player1]

    return player1_options, player2_options


# Define the callback to update the radar plot
@app.callback(
    Output('radar-plot', 'figure'),
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_radarplot(player1, player2):
    if player1 is None or player2 is None:
        return go.Figure()

    player1_stats = player_stats[player_stats['Name'] == player1].iloc[0]
    player2_stats = player_stats[player_stats['Name'] == player2].iloc[0]

    categories = player1_stats.index[2:-1]  # Adjust index to match your data structure, excluding the Photo column
    player1_values = player1_stats[2:-1].values
    player2_values = player2_stats[2:-1].values

    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=player1_values,
        theta=categories,
        fill='toself',
        name=player1
    ))

    fig.add_trace(go.Scatterpolar(
        r=player2_values,
        theta=categories,
        fill='toself',
        name=player2
    ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )
        ),
        showlegend=True,
        title=f'Comparison: {player1} vs {player2}'
    )

    return fig

# Define the callback to update the scatter plot to compare player with all the players in the dataset
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value'),
     Input('scatter-x-axis', 'value'),
     Input('scatter-y-axis', 'value')]
)
def update_scatterplot(player1, player2, xaxis, yaxis):
    if player1 is None or player2 is None or xaxis is None or yaxis is None:
        return go.Figure()

    player1_stats = player_stats[player_stats['Name'] == player1].iloc[0]
    player2_stats = player_stats[player_stats['Name'] == player2].iloc[0]

    player1_x = player1_stats[xaxis]
    player1_y = player1_stats[yaxis]
    player2_x = player2_stats[xaxis]
    player2_y = player2_stats[yaxis]

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=player_stats[xaxis],
        y=player_stats[yaxis],
        mode='markers',
        marker=dict(
            size=12,
            color=player_stats[xaxis],
            colorscale='Viridis',
            showscale=True
        ),
        text=player_stats['Name'],
        name='          All Players'
    ))
    fig.add_trace(go.Scatter(
        x=[player1_x],
        y=[player1_y],
        mode='markers',
        marker=dict(
            size=12,
            color='red'
        ),
        text=[player1],
        name='          '+player1
    ))

    fig.add_trace(go.Scatter(
        x=[player2_x],
        y=[player2_y],
        mode='markers',
        marker=dict(
            size=12,
            color='blue'
        ),
        text=[player2],
        name='          '+player2
    ))

    fig.update_layout(
        title='Player Comparison',
        xaxis_title=xaxis,
        yaxis_title=yaxis
    )

    return fig

# Define the callback to update the boxplot
@app.callback(
    [Output('boxplot-player1', 'figure'),
     Output('boxplot-player2', 'figure')],
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value'),
     Input('stat-dropdown', 'value')]
)
def update_boxplots(player1, player2, selected_stat):
    if player1 is None or player2 is None or selected_stat is None:
        return go.Figure(), go.Figure()

    # Filter stats for the selected players
    player1_stats = player_df[player_df['Name'] == player1].iloc[0]
    player2_stats = player_df[player_df['Name'] == player2].iloc[0]

    # Ensure the 'Position' column name matches your CSV
    player1_position = player1_stats['Position']
    player2_position = player2_stats['Position']

    # Filter stats for players in the same positions separately
    similar_position1_stats = player_df[player_df['Position'] == player1_position]
    similar_position2_stats = player_df[player_df['Position'] == player2_position]

    # Initialize the figures
    fig1 = go.Figure()
    fig2 = go.Figure()

    # Add box plot for the similar position players of player1
    fig1.add_trace(go.Box(
        y=similar_position1_stats[selected_stat],
        x=[player1_position] * len(similar_position1_stats),
        name=f'{player1_position} Players',
        marker_color='lightgrey',
        boxmean='sd',  # Display standard deviation
        jitter=0.3,
        pointpos=-1.8,
        boxpoints='all'
    ))

    # Add scatter plot for the selected player1
    fig1.add_trace(go.Scatter(
        y=[player1_stats[selected_stat]],
        x=[player1_position],
        name=f'{player1} - {selected_stat}',
        mode='markers',
        marker=dict(color='red', size=10, symbol='circle')
    ))

    # Update layout for fig1
    fig1.update_layout(
        title=f'{selected_stat} Distribution for {player1} and {player1_position} Players',
        xaxis_title='Position',
        yaxis_title=selected_stat,
        boxmode='group'
    )

    # Add box plot for the similar position players of player2
    fig2.add_trace(go.Box(
        y=similar_position2_stats[selected_stat],
        x=[player2_position] * len(similar_position2_stats),
        name=f'{player2_position} Players',
        marker_color='lightgrey',
        boxmean='sd',  # Display standard deviation
        jitter=0.3,
        pointpos=-1.8,
        boxpoints='all'
    ))

    # Add scatter plot for the selected player2
    fig2.add_trace(go.Scatter(
        y=[player2_stats[selected_stat]],
        x=[player2_position],
        name=f'{player2} - {selected_stat}',
        mode='markers',
        marker=dict(color='blue', size=10, symbol='circle')
    ))

    # Update layout for fig2
    fig2.update_layout(
        title=f'{selected_stat} Distribution for {player2} and {player2_position} Players',
        xaxis_title='Position',
        yaxis_title=selected_stat,
        boxmode='group'
    )

    return fig1, fig2

# Define the callback to update the header text based on player1 and player2 selection
@app.callback(
    [Output('player1-header', 'children'),
     Output('player2-header', 'children')],
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_headers(player1, player2):
    player1_header = f"{player1}'s Stats" if player1 else "Player 1 Stats"
    player2_header = f"{player2}'s Stats" if player2 else "Player 2 Stats"
    return player1_header, player2_header

# Define the callback to update DataTables based on player1 and player2 selection
@app.callback(
    [Output('stats-table1', 'data'),
     Output('stats-table2', 'data')],
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_tables(player1, player2):
    player1_data = []
    player2_data = []

    if player1:
        player1_stats = player_stats[player_stats['Name'] == player1].iloc[0]
        player1_data = [{"stat": stat, "value": player1_stats[stat]} for stat in player1_stats.index[2:-1]]
    
    if player2:
        player2_stats = player_stats[player_stats['Name'] == player2].iloc[0]
        player2_data = [{"stat": stat, "value": player2_stats[stat]} for stat in player2_stats.index[2:-1]]

    return player1_data, player2_data

# Define the callback to update player images based on selection
@app.callback(
    [Output('player1-image', 'src'),
     Output('player2-image', 'src')],
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_images(player1, player2):
    default_image1 = 'https://cdn.sofifa.net/players/158/023/22_60.png'
    default_image2 = 'https://cdn.sofifa.net/players/020/801/22_60.png'

    player1_image = default_image1
    player2_image = default_image2

    if player1:
        player1_image_url = player_df[player_df['Name'] == player1]['Photo'].values[0]
        if pd.notna(player1_image_url):
            player1_image = player1_image_url
    
    if player2:
        player2_image_url = player_df[player_df['Name'] == player2]['Photo'].values[0]
        if pd.notna(player2_image_url):
            player2_image = player2_image_url

    return player1_image, player2_image

# Define the callback to update Left card and right card values to red and green based on higher value of the two players with a exception of overall and club
@app.callback(
    [Output('stats-table1', 'style_data_conditional'),
     Output('stats-table2', 'style_data_conditional')],
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_table_colors(player1, player2):
    if player1 is None or player2 is None:
        return [], []

    player1_stats = player_stats[player_stats['Name'] == player1].iloc[0]
    player2_stats = player_stats[player_stats['Name'] == player2].iloc[0]

    style_data_conditional1 = []
    style_data_conditional2 = []

    for stat in player1_stats.index[2:-1]:
        if stat in ['Overall', 'Club']:
            continue

        player1_value = player1_stats[stat]
        player2_value = player2_stats[stat]

        if player1_value > player2_value:
            style_data_conditional1.append({
                'if': {'filter_query': f'{{stat}} = "{stat}"'},
                'backgroundColor': '#d1f7cf'
            })
            style_data_conditional2.append({
                'if': {'filter_query': f'{{stat}} = "{stat}"'},
                'backgroundColor': '#f7d1d1'
            })
        elif player1_value < player2_value:
            style_data_conditional1.append({
                'if': {'filter_query': f'{{stat}} = "{stat}"'},
                'backgroundColor': '#f7d1d1'
            })
            style_data_conditional2.append({
                'if': {'filter_query': f'{{stat}} = "{stat}"'},
                'backgroundColor': '#d1f7cf'
            })

    return style_data_conditional1, style_data_conditional2

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
